In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('seaborn')
from datetime import datetime
import json
import requests
from github import Github

In [2]:
def toUnixTime(date, format):
    t2 = datetime.strptime(date, format)
    t1 = datetime(1970, 1, 1)
    ans = (t2 - t1).total_seconds()*1000
    ans = int(ans)
    return ans

In [3]:
sources = {}
data = {}

In [4]:
url = lambda metric: f'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_{metric}_global.csv' 
metrics = ['confirmed', 'recovered', 'deaths'] 
for metric in metrics: sources[metric] = url(metric)
time_series = {metric: pd.read_csv(sources[metric]) for metric in metrics}

for metric in time_series:
    df = time_series[metric]
    gb = df.groupby('Country/Region')
    df = gb.sum()
    df = df.loc[:,'1/22/20':] 
    time_series[metric] = df

metrics.append('infected')
time_series['infected'] = time_series['confirmed']-time_series['recovered']-time_series['deaths']

In [19]:
time_series['confirmed']

,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,...,2/2/21,2/3/21,2/4/21,2/5/21,2/6/21,2/7/21,2/8/21,2/9/21,2/10/21,2/11/21
Country/Region,,,,,,,,,,,,,,,,,,,,,
Afghanistan,0,0,0,0,0,0,0,0,0,0,...,55121,55174,55231,55265,55330,55335,55359,55384,55402,55420
Albania,0,0,0,0,0,0,0,0,0,0,...,79934,80941,81993,83082,84212,85336,86289,87528,88671,89776
Algeria,0,0,0,0,0,0,0,0,0,0,...,107841,108116,108381,108629,108629,109088,109313,109559,109782,110049
Andorra,0,0,0,0,0,0,0,0,0,0,...,10017,10070,10137,10172,10206,10251,10275,10312,10352,10391
Angola,0,0,0,0,0,0,0,0,0,0,...,19900,19937,19996,20030,20062,20086,20112,20163,20210,20261
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
West Bank and Gaza,0,0,0,0,0,0,0,0,0,0,...,159956,160426,161087,161559,162029,162601,163213,163975,164736,165535
Yemen,0,0,0,0,0,0,0,0,0,0,...,2122,2122,2122,2124,2127,2127,2131,2131,2133,2134
Zambia,0,0,0,0,0,0,0,0,0,0,...,56233,57489,59003,60427,61427,62633,63573,64610,65573,66598


In [5]:
# time_series['infected'] = time_series['confirmed']-time_series['recovered']-time_series['deaths']

# for metric in time_series:
#     time_series[metric].loc['World'] = time_series[metric].sum()

# for metric in metrics:
#     time_series[f'daily_{metric}'] = time_series[metric].diff(axis=1)
#     time_series[f'7MA_daily_{metric}'] = time_series[f'daily_{metric}'].rolling(window=7, axis=1).mean()

In [6]:
sources['iso'] = 'https://raw.githubusercontent.com/lukes/ISO-3166-Countries-with-Regional-Codes/master/all/all.csv'
iso_df = pd.read_csv(sources['iso'], index_col='name')
rename = {
    "Bolivia (Plurinational State of)": "Bolivia",
    "Brunei Darussalam": "Brunei",
    "Côte d'Ivoire": "Cote d'Ivoire",
    "Iran (Islamic Republic of)": "Iran",
    "Korea, Republic of": "Korea, South",
    "Taiwan, Province of China": "Taiwan*",
    "United States of America": "US",
    "Russian Federation":"Russia",
    "Venezuela (Bolivarian Republic of)": "Venezuela",
    "United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
    "Moldova, Republic of": "Moldova",
    "Viet Nam": "Vietnam",
    "Tanzania, United Republic of": "Tanzania",
    "Palestine, State of": "West Bank and Gaza",
    "Syrian Arab Republic": "Syria",
    "Lao People's Democratic Republic": "Laos",
    "Myanmar":"Burma",
    "Congo, Democratic Republic of the": "Congo (Kinshasa)",
    "Congo": "Congo (Brazzaville)"
}
iso_df.rename(index=rename, inplace=True)
iso_df.loc['Kosovo'] = 'XK'
iso_df.loc['Namibia'] = 'NA'
iso_df.loc['World'] = 'WD'

In [24]:
sources['vaccines'] = 'https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/vaccinations/vaccinations.csv'
df = pd.read_csv(sources['vaccines'])
replace = {'United States': 'US'}
df['location'].replace(replace, inplace=True)
df['date'] = df['date'].apply(lambda date: datetime.strptime(date, '%Y-%m-%d').strftime('%#m/%#d/%y')) 
vaccines_df = pd.DataFrame(index=time_series['confirmed'].index, columns=time_series['confirmed'].columns)

for country in iso_df.index:
    try:
        tmp_df = df[df['location'] == country]
        tmp_df.set_index('date', inplace=True)
        vaccines_df.loc[country] = tmp_df['total_vaccinations']
    except: continue

vaccines_df = vaccines_df.iloc[:,:-1]
vaccines_df.fillna(method='ffill', inplace=True, axis=1)
vaccines_df.fillna(value=0, inplace=True)
time_series['vaccines'] = vaccines_df

In [25]:
vaccines_df

,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,...,2/1/21,2/2/21,2/3/21,2/4/21,2/5/21,2/6/21,2/7/21,2/8/21,2/9/21,2/10/21
Country/Region,,,,,,,,,,,,,,,,,,,,,
Afghanistan,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Albania,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,549.0,550.0,550.0,550.0,550.0,550.0,550.0,550.0,1127.0,1127.0
Algeria,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,30.0,30.0,30.0,30.0,30.0,30.0,30.0,30.0,30.0,30.0
Andorra,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1036.0,1036.0,1036.0,1036.0,1036.0,1036.0,1036.0,1036.0,1036.0,1291.0
Angola,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
United States Minor Outlying Islands,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Virgin Islands (British),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Virgin Islands (U.S.),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
for metric in metrics: 
    time_series[metric].loc['World'] = time_series[metric].sum()

metrics.append("vaccines")
for metric in metrics:
    time_series[f'daily_{metric}'] = time_series[metric].diff(axis=1)
    time_series[f'7MA_daily_{metric}'] = time_series[f'daily_{metric}'].rolling(window=7, axis=1).mean()

In [9]:
general_df = pd.DataFrame(index=time_series['confirmed'].index)

for metric in metrics:
    general_df[metric] = time_series[metric].iloc[:,-1]
    general_df[f'daily_{metric}'] = time_series[f'daily_{metric}'].iloc[:,-1]

# general_df.fillna(value=0, inplace=True)
general_df = general_df.astype(int)
general_df.sort_values('confirmed', ascending=False, inplace=True) 
general_df = general_df.applymap(lambda x: '{:,}'.format(x))

In [10]:
general_df['country'] = general_df.index
general_df['iso'] = iso_df['alpha-2']
general_df['region'] = iso_df['region']
general_df['last_update'] = str(datetime.utcnow())[:-7]

In [11]:
no_match = general_df[general_df['iso'].isnull()].index
for metric in time_series:
    time_series[metric].drop(index=no_match, inplace=True)
general_df.drop(index=no_match, inplace=True)

In [12]:
class countryData:
    def __init__(self, country):
        self.general = general_df.loc[country]
        self.time_series = {metric: time_series[metric].loc[country] for metric in time_series}
        self.preProcessing()
    
    def preProcessing(self):
        def getStart(metric, atleast=1):
            s = self.time_series[metric]
            tmp_s = s[s > atleast]
            if len(tmp_s): start = tmp_s.index[0]
            else: start = s.index[0]
            if metric == '7MA_daily_confirmed' and start < '1/29/20': start = '1/29/20' 
            return start
        
        # start = getStart(metric='7MA_daily_confirmed', atleast=1000)
        start = '6/1/20'
        self.time_series = {metric: self.time_series[metric][start:] for metric in self.time_series}

        start_vaccines = getStart(metric='7MA_daily_vaccines', atleast=1000)
        self.time_series['vaccines'] = self.time_series['vaccines'][start_vaccines:]
        self.time_series['daily_vaccines'] = self.time_series['daily_vaccines'][start_vaccines:]
        self.time_series['7MA_daily_vaccines'] = self.time_series['7MA_daily_vaccines'][start_vaccines:]
        # self.time_series['starts'] = start_vaccines
        self.time_series['starts'] = {'confirmed':toUnixTime(start, format="%m/%d/%y"),'vaccines': toUnixTime(start_vaccines, format="%m/%d/%y")} 

    def to_dict(self):
        res = {
            'general': self.general.to_dict(),
            'time_series': {metric: self.time_series[metric].to_list() for metric in self.time_series if metric != 'starts'}
        }
        res['time_series']['starts'] = self.time_series['starts']
        return res

In [18]:
country_data = countryData('Portugal')
country_data.time_series['recovered']

10/10/20     52803
10/11/20     53187
10/12/20     53498
10/13/20     54047
10/14/20     54493
             ...  
2/7/21      606166
2/8/21      612921
2/9/21      628078
2/10/21     636859
2/11/21     645122
Name: Portugal, Length: 125, dtype: int64

In [14]:
def genCountryData(country):
    data = countryData(country)
    return data.to_dict()

In [15]:
def manualUpdate():
    general_df.to_json('./data/general.json', orient='records')
    for country in general_df.index:
        country_iso = general_df.loc[country]['iso']
        res = genCountryData(country)
        with open('./data/time_series/'+country_iso+'.json', 'w') as doc: json.dump(res, doc)

In [16]:
def updateData(access_token):
    g = Github(access_token)
    repo = g.get_user().get_repo("CoronaTrack")
    
    res = general.to_json(orient='records')
    contents = repo.get_contents(f"data/general.json")
    repo.update_file(contents.path, "automatic update", res, contents.sha)
    
    for country in general_df.index:
        country_iso = general_df.loc[country]['iso']
        country_data = genCountryData(country)
        res = json.dumps(country_data)
    
        contents = repo.get_contents(f"data/time_series/{country_iso}.json")
        repo.update_file(contents.path, "automatic update", res, contents.sha)


In [17]:
manualUpdate()